### 연간 엔터사별 Top 앨범 출시에 따른 주가 변동률 분석

- 일간 변동률 = daily rate of change (drc)
- 변동률 누적 합계 = cumulative sum rate of change (cs)

In [1]:
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
from calculation_of_stock_by_release_date import create_top_album_stock, calculate_rate_of_change

In [2]:
yearly_top_album_df = pd.read_csv('data/yearly_top_album_df.csv')

In [36]:
yearly_top_album_df.loc[6]

agency_name                                      SM
year                                           2020
ranking                                           4
singer                                          NCT
title           NCT RESONANCE Pt. 1 - The 2nd Album
sales                                       1289594
release_date                             2020.10.12
Name: 6, dtype: object

In [37]:
# 분석할 출시일
release_date = '2020.10.12'

# Top 앨범 출시일 이후 10 거래일 간의 주가 데이터 추출
conn = pymysql.connect(host='127.0.0.1'
            , port=3306
            , user = 'root'
            , password='root1234'
            , db = 'stock_pj'
            , charset='utf8'
            ) 

cur = conn.cursor()

cur.execute(f'''
    select * from agency_stock
    where date >= '{release_date}'
    ORDER BY date
    LIMIT 30;   
''')

top_album_stock_tuple = cur.fetchall()

# create_top_album_stock
top_album_stock_df, date_list = create_top_album_stock(top_album_stock_tuple)

# 주가 변동률 계산
rate_of_change_df = calculate_rate_of_change(top_album_stock_df, date_list)
rate_of_change_df

,date,jyp_drc,jyp_cs,sm_drc,sm_cs,yg_drc,yg_cs
0,2020-10-12,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-10-13,4.644809,4.644809,2.623907,2.623907,1.039501,1.039501
2,2020-10-14,-1.808067,2.836742,-0.439239,2.184668,0.103842,1.143343
3,2020-10-15,-5.580029,-2.743287,-7.221350,-5.036682,-7.238307,-6.094964
4,2020-10-16,1.447178,-1.296109,-3.241491,-8.278173,-2.394527,-8.489491
5,2020-10-19,-3.134328,-4.430438,-1.480263,-9.758436,-2.813599,-11.303090
6,2020-10-20,-0.299401,-4.729839,0.000000,-9.758436,0.698487,-10.604603
7,2020-10-21,1.183432,-3.546407,2.564103,-7.194334,-0.821596,-11.426200
8,2020-10-22,-3.840246,-7.386653,-4.522613,-11.716947,-2.158273,-13.584473
9,2020-10-23,0.610687,-6.775966,-0.844595,-12.561541,0.595948,-12.988525


2018년도 SM의 Top 앨범 출시일에 따른 주가 변동성은 SM이 제일 높다

----

In [129]:
# 만약 sm일떄 sm 주가변동률이 가장 높이면 True 아니면 False
# 얼마나 더 높을까? 평균 구하기
agency_name_list = []
year_list = []
jyp_cs_list = []
sm_cs_list = []
yg_cs_list = []

for i in range(len(yearly_top_album_df)):
    agency_name_list.append(yearly_top_album_df['agency_name'][i])
    year_list.append(yearly_top_album_df['year'][i])
    release_date = yearly_top_album_df['release_date'][i]

    # Top 앨범 출시일 이후 10 거래일 간의 주가 데이터 추출
    conn = pymysql.connect(host='127.0.0.1'
            , port=3306
            , user = 'root'
            , password='root1234'
            , db = 'stock_pj'
            , charset='utf8'
            ) 

    cur = conn.cursor()

    cur.execute(f'''
        select * from agency_stock
        where date >= '{release_date}'
        ORDER BY date
        LIMIT 30;   
    ''')

    top_album_stock_tuple = cur.fetchall()

    # create_top_album_stock
    top_album_stock_df, date_list = create_top_album_stock(top_album_stock_tuple)

    # 주가 변동률 계산
    rate_of_change_df = calculate_rate_of_change(top_album_stock_df, date_list)

    # 엔터사별 누적 주가 변동률 추가
    jyp_cs_list.append(rate_of_change_df.loc[9, 'jyp_cs'])                                                  
    sm_cs_list.append(rate_of_change_df.loc[9, 'sm_cs'])                                                  
    yg_cs_list.append(rate_of_change_df.loc[9, 'yg_cs'])                                                  


cs_df = pd.DataFrame()
cs_df['agency_name'] = agency_name_list
cs_df['year'] = year_list
cs_df['jyp_cs'] = jyp_cs_list
cs_df['sm_cs'] = sm_cs_list
cs_df['yg_cs'] = yg_cs_list
cs_df.shape

(15, 5)

In [130]:
cs_df

,agency_name,year,jyp_cs,sm_cs,yg_cs
0,SM,2018,8.222244,11.843308,7.760163
1,JYP,2018,1.071783,-9.207872,-0.964885
2,YG,2018,2.230056,1.186451,14.925796
3,SM,2019,-2.541081,-4.961428,2.215234
4,JYP,2019,-3.525894,-5.555336,0.528749
5,YG,2019,-6.245536,1.654020,-1.835430
6,SM,2020,-6.775966,-12.561541,-12.988525
7,YG,2020,-8.850483,-11.260996,-21.287346
8,JYP,2020,-7.897990,-5.566956,-0.508599
9,SM,2021,6.036600,17.440384,16.578404


In [131]:
agency_idx = {'JYP':2, 'SM':3, 'YG':4}
true_false_list = []

for i in range(len(cs_df)):
    idx_list = list(agency_idx.values())
    idx = agency_idx[cs_df.loc[i][0]]
    idx_list.remove(idx)
    true_false_list.append((cs_df.loc[i][idx] >= cs_df.loc[i][idx_list[0]]) or (cs_df.loc[i][idx] >= cs_df.loc[i][idx_list[1]])) # or

cs_df['result'] = true_false_list
cs_df

,agency_name,year,jyp_cs,sm_cs,yg_cs,result
0,SM,2018,8.222244,11.843308,7.760163,True
1,JYP,2018,1.071783,-9.207872,-0.964885,True
2,YG,2018,2.230056,1.186451,14.925796,True
3,SM,2019,-2.541081,-4.961428,2.215234,False
4,JYP,2019,-3.525894,-5.555336,0.528749,True
5,YG,2019,-6.245536,1.654020,-1.835430,True
6,SM,2020,-6.775966,-12.561541,-12.988525,True
7,YG,2020,-8.850483,-11.260996,-21.287346,False
8,JYP,2020,-7.897990,-5.566956,-0.508599,False
9,SM,2021,6.036600,17.440384,16.578404,True


In [132]:
true_cnt = cs_df.loc[cs_df['result'] == True, 'result'].count()
false_cnt = cs_df.loc[cs_df['result'] == False, 'result'].count()
print(f"True: {true_cnt}")
print(f"False: {false_cnt}")
print(f"True 비율: {true_cnt/15 * 100} %")

True: 9
False: 6
True 비율: 60.0 %


In [133]:
agency_idx = {'JYP':2, 'SM':3, 'YG':4}
true_false_list = []

for i in range(len(cs_df)):
    idx_list = list(agency_idx.values())
    idx = agency_idx[cs_df.loc[i][0]]
    idx_list.remove(idx)
    true_false_list.append((cs_df.loc[i][idx] >= cs_df.loc[i][idx_list[0]]) and (cs_df.loc[i][idx] >= cs_df.loc[i][idx_list[1]])) # and 

cs_df['result'] = true_false_list

true_cnt = cs_df.loc[cs_df['result'] == True, 'result'].count()
false_cnt = cs_df.loc[cs_df['result'] == False, 'result'].count()
print(f"True: {true_cnt}")
print(f"False: {false_cnt}")
print(f"True 비율: {true_cnt/15 * 100} %")

True: 5
False: 10
True 비율: 33.33333333333333 %
